In [8]:
import pandas as pd
import os.path
import datetime
from pandas import datetime

In [13]:
os.getcwd()
os.chdir('DSRCompTwo')

In [93]:
# Data Loading
df = pd.read_csv('0.Raw_data/train/Combined_News_DJIA_train.csv')   # please check if Training data is in the same location on your PC
df = df.set_index("Date")
def parser(x):
    return datetime.strptime(x, '%Y-%m-%d')

series = pd.read_csv('0.Raw_data/DJIA_table.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
series = series.sort_index(ascending=True)
print(series.head())

                    Open          High           Low         Close     Volume  \
Date                                                                            
2008-08-08  11432.089844  11759.959961  11388.040039  11734.320312  212830000   
2008-08-11  11729.669922  11867.110352  11675.530273  11782.349609  183190000   
2008-08-12  11781.700195  11782.349609  11601.519531  11642.469727  173590000   
2008-08-13  11632.809570  11633.780273  11453.339844  11532.959961  182550000   
2008-08-14  11532.070312  11718.280273  11450.889648  11615.929688  159790000   

               Adj Close  
Date                      
2008-08-08  11734.320312  
2008-08-11  11782.349609  
2008-08-12  11642.469727  
2008-08-13  11532.959961  
2008-08-14  11615.929688  


In [94]:

series['lag_adj_close'] = series['Adj Close'].shift(1)
series['diff_close'] = series['Adj Close']-series['lag_adj_close']
series['target'] = 0
series['target'] = (series['diff_close']>=0 ).map(int)
#series['target'] = diff
series['target']
series_new = series.loc[:'2014-11-20']
new_df=pd.concat([series_new['target'],df['Label']],axis=1)

In [96]:
df['Label']  = series_new['target']

In [104]:
df.index

Index(['2008-08-08', '2008-08-11', '2008-08-12', '2008-08-13', '2008-08-14',
       '2008-08-15', '2008-08-18', '2008-08-19', '2008-08-20', '2008-08-21',
       ...
       '2014-11-07', '2014-11-10', '2014-11-11', '2014-11-12', '2014-11-13',
       '2014-11-14', '2014-11-17', '2014-11-18', '2014-11-19', '2014-11-20'],
      dtype='object', name='Date', length=1584)

In [105]:
# Data Cleaning and merging all headlines to one single column

subdf = df.iloc[:,2:27] 
subdf = subdf.applymap(str)
s = subdf.apply(lambda x: ' '.join(x), axis=1)

# replace the b' and b" which are in the beginning of some headlines
s = s.str.replace("b'","")
s = s.str.replace('b"','')

# create a new dataframe with all headlines and the overall word count
df1 = s.to_frame(name='headlines')
df1['lengths'] = df1['headlines'].apply(len)
df1['date'] = df.index

In [108]:
len(df1)

1584

In [111]:
val_size = 0.2
train_length = int(len(df1)*(1-val_size))
train = df1.iloc[:train_length+1,:]
val = df1.iloc[train_length:,:]

In [114]:
print(len(train))
print(len(val))
#split dataset base don time
X_train = train.loc[:,]
X_val = val.loc[:,]
y_train = df.iloc[:train_length+1,:]
y_train=y_train['Label']
y_val = df.iloc[train_length:,:]
y_val=y_val['Label']

1268
317


In [115]:
y_val.shape

(317,)

In [116]:
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import pickle


# create function to clean the text
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Lower case of all words
    2. Remove all punctuation
    3. Remove all stopwords
    4. Returns a list of the cleaned text
    """

    # transforms all to lower case words
    mess = mess.lower()

    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)

    # Now just remove any stopwords
    return [word for word in nopunc.split() if word not in stopwords.words('english')]

# create a WordNet tokenizer
wrd_lemmatizer = WordNetLemmatizer()

def wn_tokenizer(doc):
    tokens = word_tokenize(doc)
    return wrd_lemmatizer.lemmatize(tokens, pos='v')

def bow_encoding(s):
    '''
    Takes a pandas Series and fits and transforms the strings to a Bag of Words matrix
    :param s: pandas series
    :return: Numpy Matrix
    '''

    BoW_WN = CountVectorizer(analyzer=text_process, tokenizer=wn_tokenizer, min_df=4)
    BoW_WN_matrix = BoW_WN.fit_transform(s)
    with open('CountVectorizer.pk', 'wb') as fin:
        pickle.dump(BoW_WN, fin)
    return BoW_WN_matrix

def bow_encoding_val(s):
    '''
    Takes a pandas Series and transforms the strings to a Bag of Words matrix
    :param s: pandas series
    :return: Numpy Matrix
    '''

    with open('CountVectorizer.pk', 'rb') as f:
        BoW_WN = pickle.load(f)
    BoW_WN_matrix = BoW_WN.transform(s)

    return BoW_WN_matrix


def tfidf_encoding(s):
    '''
    Takes a series of Texts and fit and transforms the strings to a TFIDF matrix
    :param s: pandas series
    :return: numpy matrix
    '''

    tfidf_vectorizer_WN = TfidfVectorizer(analyzer=text_process, use_idf=True, tokenizer=wn_tokenizer, min_df=4)
    tfidf_WN_matrix = tfidf_vectorizer_WN.fit_transform(s)
    with open('TFIDFVectorizer.pk', 'wb') as fin:
        pickle.dump(tfidf_vectorizer_WN, fin)
    return tfidf_WN_matrix


def tfidf_encoding_val(s):
    '''
    Takes a series of Texts and transforms the strings to a TFIDF matrix
    :param s: pandas series
    :return: Numpy Matrix
    '''

    with open('TFIDFVectorizer.pk', 'rb') as f:
        tfidf_vectorizer_WN = pickle.load(f)
    tfidf_WN_matrix = tfidf_vectorizer_WN.transform(s)

    return tfidf_WN_matrix

In [118]:
# Encoding the training data set
df_encode = bow_encoding(X_train['headlines'])
df_encode

<1268x9967 sparse matrix of type '<class 'numpy.int64'>'
	with 268532 stored elements in Compressed Sparse Row format>

In [119]:
# transforming
#from BoW import bow_encoding_val
df_encode_val = bow_encoding_val(X_val['headlines'])
df_encode_val

<317x9967 sparse matrix of type '<class 'numpy.int64'>'
	with 68758 stored elements in Compressed Sparse Row format>

In [121]:
df_encode_TFIDF = tfidf_encoding(X_train['headlines'])
df_encode_TFIDF

<1268x9967 sparse matrix of type '<class 'numpy.float64'>'
	with 268532 stored elements in Compressed Sparse Row format>

In [122]:
df_encode_TFIDF_val = tfidf_encoding_val(X_val['headlines'])
df_encode_TFIDF_val

<317x9967 sparse matrix of type '<class 'numpy.float64'>'
	with 68758 stored elements in Compressed Sparse Row format>

In [123]:
X_bow_train= pd.DataFrame(df_encode.toarray())
X_bow_val= pd.DataFrame(df_encode_val.toarray())
X_TFIDF_train= pd.DataFrame(df_encode_TFIDF.toarray())
X_TFIDF_val= pd.DataFrame(df_encode_TFIDF_val.toarray())

In [124]:
Y_train = y_train
Y_val = y_val

In [125]:
import os
os.getcwd()
#os.chdir('..')

'C:\\Users\\lundr\\DSRCompTwo'

In [126]:
X_bow_train.to_pickle('4.Data/X_bow_train_v3.pkl')
X_bow_val.to_pickle('4.Data/X_bow_val_v3.pkl')
X_TFIDF_train.to_pickle('4.Data/X_TFIDF_train_v3.pkl')
X_TFIDF_val.to_pickle('4.Data/X_TFIDF_val_v3.pkl')



In [127]:
Y_train.to_pickle('4.Data/Y_train_v3.pkl')
Y_val.to_pickle('4.Data/Y_val_v3.pkl')